In [1]:
#credits: https://www.kaggle.com/paultimothymooney for the dataset
#credits: https://www.kaggle.com/aakashnain for the model

!pip install flask_ngrok
!pip install bokeh

import pandas as pd
import numpy as np
import os
import keras
import random
import matplotlib.pyplot as plt
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from keras.optimizers import Adam
from imageio import imread
from PIL import Image
from werkzeug.utils import secure_filename
from flask import Flask, flash, render_template, request, redirect, url_for, send_file
from flask_ngrok import run_with_ngrok
from google.colab import files
from bokeh.plotting import figure
from bokeh.embed import components
from numpy import pi, squeeze

Using TensorFlow backend.


In [2]:
# Mount our Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Loading our model from Google Drive
model = load_model("/content/drive/My Drive/pneumonia/chest-xray-pneumonia.h5")

# Output a [0, 1] score for being normal or ill
def pred_img(pic):
    img = image.load_img(pic, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    normal_score, pneumonia_score = np.around(model.predict(x), decimals=2)[0]
    return {'normal_score': normal_score, 'pneumonia_score': pneumonia_score}

# Testing our method
ill_path = "/content/drive/My Drive/pneumonia/chest_xray/test/PNEUMONIA/"
good_path = "/content/drive/My Drive/pneumonia/chest_xray/test/NORMAL/"
ill_pic = ill_path + os.listdir(ill_path)[3]
good_pic = good_path + os.listdir(good_path)[3]

print(pred_img(ill_pic))
print(pred_img(good_pic))













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


{'p_good': 0.0, 'p_ill': 1.0}
{'p_good': 1.0, 'p_ill': 0.0}


In [4]:
# Setting up templates and upload folder
!cp -r '/content/drive/My Drive/pneumonia/templates' '/content/'
!mkdir uploads

mkdir: cannot create directory ‘uploads’: File exists


In [5]:
''' Our Flask web app '''
app = Flask(__name__)

# Secret key for image upload
app.config['SECRET_KEY'] = "muchsecret"

# Upload folder path
app.config['UPLOAD_FOLDER'] = '/content/uploads/'

# Start ngrok when app is run
run_with_ngrok(app)  

# Definitions
ALLOWED_EXTENSIONS = set(['png', 'bmp', 'jpg', 'jpeg', 'gif'])
LETTER_SET = list(set('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'))
IMAGE_LABELS = ['Normal', 'Pneumonia']

# Check if a filename's extension is acceptable
def is_allowed_file(filename):
    allowed_ext = filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
    return '.' in filename and allowed_ext

# Generate a random name for an uploaded file.
def generate_random_name(filename):
    ext = filename.split('.')[-1]
    rns = [random.randint(0, len(LETTER_SET) - 1) for _ in range(3)]
    chars = ''.join([LETTER_SET[rn] for rn in rns])

    new_name = "{new_fn}.{ext}".format(new_fn=chars, ext=ext)
    new_name = secure_filename(new_name)

    return new_name

# Generates script and bar plot of predictions
def generate_barplot(predictions):
    plot = figure(x_range=IMAGE_LABELS, plot_height=300, plot_width=400)
    plot.vbar(x=IMAGE_LABELS, top=predictions, width=0.8)
    plot.xaxis.major_label_orientation = pi / 2.

    return components(plot)

# Loads and prepares the image
def load_and_prepare(filepath):
    img = image.load_img(filepath, target_size=(224, 224))
    image_data = image.img_to_array(img)
    image_data = np.expand_dims(image_data, axis=0)
    image_data = preprocess_input(image_data)
    return image_data

# Scale the image to the correct dimensions
def make_thumbnail(filepath):
    img = Image.open(filepath)
    thumb = None
    w, h = img.size

    # If it is exactly 512x512, do nothing
    if w == 512 and h == 512:
        return True

    # If the width and height are equal, scale down
    if w == h:
        thumb = img.resize((512, 512), Image.BICUBIC)
        thumb.save(filepath)
        return True

    # When the image's width is smaller than the height
    if w < h:
        # Scale so that the width is 512
        ratio = w / 512.
        w_new, h_new = 512, int(h / ratio)
        thumb = img.resize((w_new, h_new), Image.BICUBIC)

        # Crop the excess
        top, bottom = 0, 0
        margin = h_new - 512
        top, bottom = margin // 2, 512 + margin // 2
        box = (0, top, 512, bottom)
        cropped = thumb.crop(box)
        cropped.save(filepath)
        return True

    # When the image's height is smaller than the width
    if h < w:
        # Scale so that the height is 512
        ratio = h / 512
        w_new, h_new = int(w / ratio), 512
        thumb = img.resize((w_new, h_new), Image.BICUBIC)

        # Crop the excess
        left, right = 0, 0
        margin = w_new - 512
        left, right = margin // 2, 512 + margin // 2
        box = (left, 0, right, 512)
        cropped = thumb.crop(box)
        cropped.save(filepath)
        return True
    return False

# Main page
@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'GET':
        # Upload form
        return render_template('home.html')
    
    if request.method == 'POST':
        # Check if file uploaded
        if 'image' not in request.files:
            flash('No file uploaded.')
            return redirect(request.url)
        
        image_file = request.files['image']
        
        # Check if filename empty
        if image_file.filename == '':
            flash('No file uploaded.')
            return redirect(request.url)
        
        # If file uploaded
        if image_file and is_allowed_file(image_file.filename):
            try:
                filename = generate_random_name(image_file.filename)
                filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
                image_file.save(filepath)
                passed = make_thumbnail(filepath)
            except Exception:
                passed = False
            
            if passed:
                return redirect(url_for('predict', filename=filename))
            else:
                flash('An error occurred, try again.')
                return redirect(request.url)

# Predict page
@app.route('/predict/<filename>', methods=['GET'])
def predict(filename):
    # Setting up image for prediction
    image_url = url_for('images', filename=filename)
    image_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    image_data = load_and_prepare(image_path)
    
    # Prediction
    normal_score, pneumonia_score = np.around(model.predict(image_data), decimals=2)[0]

    # Setting up bar plot
    predictions = [normal_score, pneumonia_score]
    script, div = generate_barplot(predictions)
    
    return render_template('predict.html', plot_script=script, plot_div=div, image_url=image_url)

# Display image page
@app.route('/images/<filename>', methods=['GET'])
def images(filename):
    return send_file(os.path.join(app.config['UPLOAD_FOLDER'], filename))

# Error page
@app.errorhandler(500)
def server_error(error):
    return render_template('error.html'), 500

# Main app
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1e60afeb.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Jan/2020 12:12:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2020 12:12:55] "GET /static/styles/app.css HTTP/1.1" 404 -
127.0.0.1 - - [15/Jan/2020 12:12:56] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Jan/2020 12:13:05] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [15/Jan/2020 12:13:06] "GET /predict/1NU.jpg HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2020 12:13:06] "GET /images/1NU.jpg HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2020 12:13:06] "GET /static/styles/app.css HTTP/1.1" 404 -
